# **Import**

In [1]:
!pip install keras-rl
!pip install music21

    100% |████████████████████████████████| 40kB 2.7MB/s 
  Running setup.py bdist_wheel for keras-rl ... - \ done
  Stored in directory: /root/.cache/pip/wheels/7d/4d/84/9254c9f2e8f51865cb0dac8e79da85330c735551d31f73c894
Successfully built keras-rl
    100% |████████████████████████████████| 18.5MB 2.2MB/s 
  Running setup.py bdist_wheel for music21 ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done
  Stored in directory: /root/.cache/pip/wheels/7b/21/95/d396f231b8095f30aba2a1fbffbc2411fb22eb4e611ddbed57
Successfully built music21


In [2]:
from google.colab import files
import numpy as np
from music21 import stream, converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import CuDNNLSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


# **Data Import**

In [3]:
uploaded = files.upload()

fileNames = [];

for fn in uploaded.keys():
  fileNames.append(fn)

Saving rac_op3_2.mid to rac_op3_2.mid
Saving rac_op23_2.mid to rac_op23_2.mid
Saving rac_op23_3.mid to rac_op23_3.mid
Saving rac_op23_5.mid to rac_op23_5.mid
Saving rac_op23_7.mid to rac_op23_7.mid
Saving rac_op32_1.mid to rac_op32_1.mid
Saving rac_op32_13.mid to rac_op32_13.mid
Saving rac_op33_5.mid to rac_op33_5.mid
Saving rac_op33_6.mid to rac_op33_6.mid
Saving rac_op33_8.mid to rac_op33_8.mid


# **Data Processing Helper Functions**

In [0]:
def fractionToNumString(n):
  n = float(n)
  n = str(n)
  return n

def fractionToNumStringWithOffset(n, offset):
  n = float(n) - float(offset)
  n = str(n)
  return n

def createTimingString(element, offset):
  tempOffset = fractionToNumStringWithOffset(element.offset, offset)
  tempDuration = fractionToNumString(element.duration.quarterLength)
  result = '/'.join([tempOffset, tempDuration])
  return result

# **Data Processing**

In [5]:
'''
Parse all notes, offsets, duration, and velocity into strings
'''
score = []
timings = []
volumes = []

for file in fileNames:
  midi = converter.parse(file)
  print("Parsing {}".format(file))
  
  offset = 0
  notes_to_parse = None
  try: 
    # file has instrument
    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse()
  except: 
    # file has notes in a flat structure
    notes_to_parse = midi.flat.notes

  for element in notes_to_parse:
    if isinstance(element, note.Note):
      score.append(str(element.pitch))
      timings.append(createTimingString(element, offset))
      volumes.append(str(element.volume.velocity))
      offset = element.offset
    elif isinstance(element, chord.Chord):
      score.append('.'.join(str(n) for n in element.normalOrder))
      timings.append(createTimingString(element, offset))
      volumes.append(str(element.volume.velocity))
      offset = element.offset

print(score)
print(timings)
print(volumes)

Parsing rac_op3_2.mid
Parsing rac_op23_2.mid
Parsing rac_op23_3.mid
Parsing rac_op23_5.mid
Parsing rac_op23_7.mid
Parsing rac_op32_1.mid
Parsing rac_op32_13.mid
Parsing rac_op33_5.mid
Parsing rac_op33_6.mid
Parsing rac_op33_8.mid
['A3', '9', 'G#3', '8', 'C#3', '1', '8.1', '4.8', '11.4', '8.11', '10.3', '7.10', 'A3', '9', '9.2', '0.6', 'G#3', '8', '0.6', '3.6', 'C#3', '1', '8.1', '4.8', '11.4', '8.11', '10.3', '7.10', 'A3', '9', '9.2', '0.6', 'G#3', '8', '0.6', '3.6', 'C#3', '1', '8.1', '4.8', '8.1', '11.1', '1.6', '9.1', 'C4', '0', '4.9', '6.9', 'B3', '11', '3.9', '6.9', 'E3', '4', '11.4', '8.11', '11.4', '3.4', '4.9', '1.4', 'E4', '4', '8.1', '10.1', 'E-4', '3', '1.7', '10.1', 'G#3', '8', '3.8', '11.3', '4.9', '1.4', '1.6', '9.1', '3.8', '11.3', '11.4', '8.11', '1.6', '9.1', '3.9', '6.9', '11.4', '8.11', '11.4', '8.11', '1.6', '9.1', '3.9', '6.9', '11.4', '8.11', '8.1', '4.8', '3.9', '6.9', '0.6', '3.6', '8.1', '4.8', '8.1', '4.8', '11.4', '8.11', '10.3', '7.10', 'A3', '9', '9.2', '0.

# **Data Preparation Helper Functions**

In [0]:
def createDictionaries(dataSet):
  types = sorted(set(item for item in dataSet))
  dictionary = dict((item, num) for num,item in enumerate(types))
  reverseDictionary = dict((num, item) for num,item in enumerate(types))
  return dictionary, reverseDictionary

def addWindowToDataSet(datasetX, datasetY, source, dictionary, offset, WINDOW):
  x = source[offset: offset + WINDOW]
  y = source[offset + WINDOW]
  datasetX.append([dictionary[c] for c in x])
  datasetY.append(dictionary[y])
  return datasetX, datasetY

def reshapeForModel(X, Y, size, WINDOW, vocab):
  X = np.reshape(X, (size, WINDOW, 1))
  X = X / float(vocab)
  Y = np_utils.to_categorical(Y)
  return X, Y

# **Data Preparation**

In [7]:
'''
Create dictionary for notes
'''
note_dictionary, reverse_note_dictionary = createDictionaries(score)
n_vocab = len(note_dictionary)

print(note_dictionary)

{'0': 0, '0.2': 1, '0.2.5': 2, '0.2.6': 3, '0.3': 4, '0.3.5': 5, '0.3.6': 6, '0.3.6.9': 7, '0.3.7': 8, '0.4': 9, '0.4.5': 10, '0.4.6': 11, '0.4.7': 12, '0.4.8': 13, '0.5': 14, '0.6': 15, '1': 16, '1.2': 17, '1.3': 18, '1.3.6': 19, '1.3.7': 20, '1.3.8': 21, '1.4': 22, '1.4.7': 23, '1.4.7.10': 24, '1.4.7.9': 25, '1.4.8': 26, '1.5': 27, '1.5.6': 28, '1.5.7': 29, '1.5.8': 30, '1.5.9': 31, '1.6': 32, '1.7': 33, '10': 34, '10.0': 35, '10.0.3': 36, '10.0.3.6': 37, '10.0.4': 38, '10.1': 39, '10.1.3': 40, '10.1.4': 41, '10.1.5': 42, '10.11': 43, '10.2': 44, '10.2.3': 45, '10.2.4': 46, '10.2.5': 47, '10.3': 48, '11': 49, '11.0': 50, '11.1': 51, '11.1.4': 52, '11.1.4.7': 53, '11.1.5': 54, '11.2': 55, '11.2.5': 56, '11.2.6': 57, '11.3': 58, '11.3.4': 59, '11.3.5': 60, '11.3.6': 61, '11.4': 62, '2': 63, '2.3': 64, '2.4': 65, '2.4.7': 66, '2.4.7.10': 67, '2.4.9': 68, '2.5': 69, '2.5.7': 70, '2.5.8': 71, '2.5.8.10': 72, '2.5.8.11': 73, '2.5.9': 74, '2.6': 75, '2.6.10': 76, '2.6.8': 77, '2.6.9': 78, '

In [8]:
'''
Create dictionary for offset and duration
'''
timing_dictionary, reverse_timing_dictionary = createDictionaries(timings)
t_vocab = len(timing_dictionary)

print(timing_dictionary)

{'0.0/0.0': 0, '0.0/0.25': 1, '0.0/0.3333333333333333': 2, '0.0/0.5': 3, '0.0/0.6666666666666666': 4, '0.0/0.75': 5, '0.0/1.0': 6, '0.0/1.25': 7, '0.0/1.3333333333333333': 8, '0.0/1.5': 9, '0.0/1.6666666666666667': 10, '0.0/1.75': 11, '0.0/10.0': 12, '0.0/2.0': 13, '0.0/2.25': 14, '0.0/2.3333333333333335': 15, '0.0/2.5': 16, '0.0/2.6666666666666665': 17, '0.0/2.75': 18, '0.0/3.0': 19, '0.0/3.75': 20, '0.0/4.0': 21, '0.0/4.5': 22, '0.0/5.0': 23, '0.0/6.0': 24, '0.0/8.0': 25, '0.0833333333333286/0.0': 26, '0.0833333333333286/0.25': 27, '0.0833333333333286/0.3333333333333333': 28, '0.0833333333333286/0.5': 29, '0.0833333333333286/0.6666666666666666': 30, '0.0833333333333286/1.0': 31, '0.0833333333333286/4.0': 32, '0.0833333333333286/4.25': 33, '0.08333333333333215/0.25': 34, '0.08333333333333304/0.25': 35, '0.08333333333333326/0.25': 36, '0.08333333333333331/0.25': 37, '0.08333333333333337/0.25': 38, '0.08333333333333348/0.25': 39, '0.08333333333333393/0.25': 40, '0.0833333333333357/0.0':

In [9]:
'''
Create dictionary for velocities
'''
velocity_dictionary, reverse_velocity_dictionary = createDictionaries(volumes)
v_vocab = len(velocity_dictionary)

print(velocity_dictionary)

{'100': 0, '101': 1, '102': 2, '103': 3, '104': 4, '105': 5, '106': 6, '107': 7, '108': 8, '113': 9, '119': 10, '12': 11, '15': 12, '16': 13, '17': 14, '18': 15, '19': 16, '20': 17, '21': 18, '22': 19, '23': 20, '24': 21, '25': 22, '26': 23, '27': 24, '28': 25, '29': 26, '30': 27, '31': 28, '32': 29, '33': 30, '34': 31, '35': 32, '36': 33, '37': 34, '38': 35, '39': 36, '40': 37, '41': 38, '42': 39, '43': 40, '44': 41, '45': 42, '46': 43, '47': 44, '48': 45, '49': 46, '50': 47, '51': 48, '52': 49, '53': 50, '54': 51, '55': 52, '56': 53, '57': 54, '58': 55, '59': 56, '60': 57, '61': 58, '62': 59, '63': 60, '64': 61, '65': 62, '66': 63, '67': 64, '68': 65, '69': 66, '70': 67, '71': 68, '72': 69, '73': 70, '74': 71, '75': 72, '76': 73, '77': 74, '78': 75, '79': 76, '80': 77, '81': 78, '82': 79, '83': 80, '84': 81, '85': 82, '86': 83, '87': 84, '88': 85, '89': 86, '90': 87, '91': 88, '92': 89, '93': 90, '94': 91, '95': 92, '96': 93, '97': 94, '98': 95, '99': 96}


In [0]:
'''
Create Sequences for Training
'''
WINDOW = 100

XNote = []
YNote = []
XTiming = []
YTiming = []
XVelocity = []
YVelocity = []

for i in range(0, len(score) - WINDOW, 1):
  XNote, YNote = addWindowToDataSet(XNote, YNote, score, note_dictionary, i, WINDOW)
  XTiming, YTiming = addWindowToDataSet(XTiming, YTiming, timings, timing_dictionary, i, WINDOW)
  XVelocity, YVelocity = addWindowToDataSet(XVelocity, YVelocity, volumes, velocity_dictionary, i, WINDOW)
  
noteDataSetSize = len(XNote)
timingDataSetSize = len(XTiming)
velocityDataSetSize = len(XVelocity)

XNote, YNote = reshapeForModel(XNote, YNote, noteDataSetSize, WINDOW, n_vocab)
XTiming, YTiming = reshapeForModel(XTiming, YTiming, timingDataSetSize, WINDOW, t_vocab)
XVelocity, YVelocity = reshapeForModel(XVelocity, YVelocity, velocityDataSetSize, WINDOW, v_vocab)

In [11]:
#For output shape checking
#Please check if all outputshapes are correct before proceeding
print(YNote.shape)
print(n_vocab)

print(YTiming.shape)
print(t_vocab)

print(YVelocity.shape)
print(v_vocab)

(13151, 266)
266
(13151, 228)
228
(13151, 97)
97


# **Note Model Definition**

In [12]:
noteModel = Sequential()

noteModel.add(CuDNNLSTM(512, input_shape=(WINDOW, 1), return_sequences=True))
noteModel.add(Dropout(0.5))
noteModel.add(CuDNNLSTM(512))
noteModel.add(Dense(256))
noteModel.add(Dropout(0.5))
noteModel.add(Dense(n_vocab))
noteModel.add(Activation('softmax'))

noteModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 100, 512)          1054720   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 512)               2101248   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 266)               68362     
_________________________________________________________________
activation_1 (Activation)    (None, 266)               0         
Total para

# **Timing Model Definition**

In [13]:
timingModel = Sequential()

timingModel.add(CuDNNLSTM(512, input_shape=(WINDOW, 1), return_sequences=True))
timingModel.add(Dropout(0.5))
timingModel.add(CuDNNLSTM(512))
timingModel.add(Dense(256))
timingModel.add(Dropout(0.5))
timingModel.add(Dense(t_vocab))
timingModel.add(Activation('softmax'))

timingModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_3 (CuDNNLSTM)     (None, 100, 512)          1054720   
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
cu_dnnlstm_4 (CuDNNLSTM)     (None, 512)               2101248   
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 228)               58596     
_________________________________________________________________
activation_2 (Activation)    (None, 228)               0         
Total para

# **Velocity Model Definition**

In [14]:
velocityModel = Sequential()

velocityModel.add(CuDNNLSTM(512, input_shape=(WINDOW, 1), return_sequences=True))
velocityModel.add(Dropout(0.5))
velocityModel.add(CuDNNLSTM(512))
velocityModel.add(Dense(256))
velocityModel.add(Dropout(0.5))
velocityModel.add(Dense(v_vocab))
velocityModel.add(Activation('softmax'))

velocityModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_5 (CuDNNLSTM)     (None, 100, 512)          1054720   
_________________________________________________________________
dropout_5 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
cu_dnnlstm_6 (CuDNNLSTM)     (None, 512)               2101248   
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 97)                24929     
_________________________________________________________________
activation_3 (Activation)    (None, 97)                0         
Total para

# **Model Training Parameters**

In [0]:
'''
Define number of epochs and batch_size for noteModel, timingModel, velocityModel
'''
noteEpochs = 200
noteBatchSize = 200
timingEpochs = 200
timingBatchSize = 200
velocityEpochs = 200
velocityBatchSize = 200

# **Note Model Training**

In [16]:
noteModel.compile(loss='categorical_crossentropy', optimizer='rmsprop')

weights_filename = 'note_model_weights.h5f'
checkpoint = ModelCheckpoint(
    weights_filename,
    monitor='loss',
    verbose=0
)
callbacks = [checkpoint]

noteModel.fit(
    XNote,
    YNote,
    epochs=noteEpochs,
    batch_size=noteBatchSize,
    callbacks=callbacks
)

Epoch 1/200
13151/13151 [==============================] - 24s 2ms/step - loss: 5.0846
Epoch 2/200
13151/13151 [==============================] - 20s 2ms/step - loss: 4.8874
Epoch 3/200
13151/13151 [==============================] - 20s 2ms/step - loss: 4.7802
Epoch 4/200
13151/13151 [==============================] - 20s 2ms/step - loss: 4.7725
Epoch 5/200
13151/13151 [==============================] - 20s 2ms/step - loss: 4.6887
Epoch 6/200
13151/13151 [==============================] - 20s 2ms/step - loss: 4.6527
Epoch 7/200
13151/13151 [==============================] - 20s 2ms/step - loss: 4.6304
Epoch 8/200
13151/13151 [==============================] - 20s 2ms/step - loss: 4.6096
Epoch 9/200
13151/13151 [==============================] - 20s 2ms/step - loss: 4.5902
Epoch 10/200
13151/13151 [==============================] - 20s 2ms/step - loss: 4.5804
Epoch 11/200
13151/13151 [==============================] - 20s 2ms/step - loss: 4.5699
Epoch 12/200
13151/13151 [===============

# **Timing Model Training**

In [17]:
timingModel.compile(loss='categorical_crossentropy', optimizer='rmsprop')

weights_filename = 'timing_model_weights.h5f'
checkpoint = ModelCheckpoint(
    weights_filename,
    monitor='loss',
    verbose=0
)
callbacks = [checkpoint]

timingModel.fit(
    XTiming,
    YTiming,
    epochs=timingEpochs,
    batch_size=timingBatchSize,
    callbacks=callbacks
)

Epoch 1/200
13151/13151 [==============================] - 20s 2ms/step - loss: 3.8914
Epoch 2/200
13151/13151 [==============================] - 20s 2ms/step - loss: 3.4209
Epoch 3/200
13151/13151 [==============================] - 20s 2ms/step - loss: 3.3914
Epoch 4/200
13151/13151 [==============================] - 20s 2ms/step - loss: 3.3773
Epoch 5/200
13151/13151 [==============================] - 20s 2ms/step - loss: 3.3708
Epoch 6/200
13151/13151 [==============================] - 20s 2ms/step - loss: 3.3610
Epoch 7/200
13151/13151 [==============================] - 20s 2ms/step - loss: 3.3579
Epoch 8/200
13151/13151 [==============================] - 20s 2ms/step - loss: 3.3545
Epoch 9/200
13151/13151 [==============================] - 20s 2ms/step - loss: 3.3555
Epoch 10/200
13151/13151 [==============================] - 20s 2ms/step - loss: 3.3502
Epoch 11/200
13151/13151 [==============================] - 20s 2ms/step - loss: 3.3564
Epoch 12/200
13151/13151 [===============

# **Velocity Model Training**

In [0]:
velocityModel.compile(loss='categorical_crossentropy', optimizer='rmsprop')

weights_filename = 'velocity_model_weights.h5f'
checkpoint = ModelCheckpoint(
    weights_filename,
    monitor='loss',
    verbose=0
)
callbacks = [checkpoint]

velocityModel.fit(
    XVelocity,
    YVelocity,
    epochs=velocityEpochs,
    batch_size=velocityBatchSize,
    callbacks=callbacks
)

Epoch 1/200
13151/13151 [==============================] - 20s 2ms/step - loss: 4.3450
Epoch 2/200
13151/13151 [==============================] - 20s 2ms/step - loss: 4.0216
Epoch 3/200
13151/13151 [==============================] - 20s 2ms/step - loss: 3.8768
Epoch 4/200
13151/13151 [==============================] - 20s 2ms/step - loss: 3.8106
Epoch 5/200
13151/13151 [==============================] - 20s 2ms/step - loss: 3.7589
Epoch 6/200
13151/13151 [==============================] - 20s 2ms/step - loss: 3.7313
Epoch 7/200
13151/13151 [==============================] - 20s 2ms/step - loss: 3.7184
Epoch 8/200
13151/13151 [==============================] - 20s 2ms/step - loss: 3.6941
Epoch 9/200
13151/13151 [==============================] - 20s 2ms/step - loss: 3.6813
Epoch 10/200
13151/13151 [==============================] - 20s 2ms/step - loss: 3.6637
Epoch 11/200
13151/13151 [==============================] - 20s 2ms/step - loss: 3.6416
Epoch 12/200
13151/13151 [===============

# **Music Generation Helper Functions**

In [0]:
def generateSequence(songlength, model, dataset, reverse_dictionary, num_vocab):
  seed = np.random.randint(0, len(dataset) - 1)
  print("seed : {0}".format(seed))
  currentSequence = dataset[seed][:]
  generatedSequence = []
  for i in range(songlength):
    x = np.reshape(currentSequence, (1, len(currentSequence), 1))
    x = x / float(num_vocab)
    p = model.predict(x, verbose=0)
    index = np.argmax(p)
    result = reverse_dictionary[index]
    generatedSequence.append(result)
    currentSequence = np.append(currentSequence, index)
    currentSequence = currentSequence[1 : len(currentSequence)]
  return generatedSequence

def createChord(sequence, timings, offset, velocity):
  notes_in_chord = sequence.split('.')
  notes = []
  for n in notes_in_chord:
    new_n = note.Note(int(n))
    new_n.storedInstrument = instrument.Piano()
    notes.append(new_n)
  new_chord = chord.Chord(notes)
  new_chord.offset = offset
  new_chord.duration.quarterLength = float(timings[1])
  new_chord.volume.velocity = velocity
  return new_chord

def createNote(sequence, timings, offset, velocity):
  new_n = note.Note(sequence)
  new_n.offset = offset
  new_n.duration.quarterLength = float(timings[1])
  new_n.volume.velocity = velocity
  new_n.storedInstrument = instrument.Piano()
  return new_n

# **Music Generation**

In [24]:
songlength = 25000

generatedNotes = generateSequence(songlength, noteModel, XNote, reverse_note_dictionary, n_vocab)
generatedTimings = generateSequence(songlength, timingModel, XTiming, reverse_timing_dictionary, t_vocab)
generatedVelocities = generateSequence(songlength, velocityModel, XVelocity, reverse_velocity_dictionary, v_vocab)

seed : 13029
seed : 3815
seed : 12051


In [25]:
'''
Convert to midi
'''
offset = 0
output_notes = []

for i in range(songlength):
  sequence = generatedNotes[i]
  timings = generatedTimings[i].split('/')
  offset += float(timings[0])
  velocity = float(generatedVelocities[i])
  
  #if sequence is chord
  if('.' in sequence) or sequence.isdigit():
    new_chord = createChord(sequence, timings, offset, velocity)
    output_notes.append(new_chord)
  #if sequence is note
  else:
    new_n = createNote(sequence, timings, offset, velocity)
    output_notes.append(new_n)
    
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='AI_ongaku2.mid')

'AI_ongaku2.mid'